# BCI-4-ALS

## Mid-Semester Project

### Project Overview

Over the mid-semester project we have been integrated
the different parts of BCI which been discussed in the course.
The project focused on Motor Imagery interface which aims
to classify between imagery of moving right hand, left hand and idle.

Over the first part in the report we present the different
codes we made for the MI classification. This part can be used
as a `README` file for next years.

In [ ]:
import yaml
import pyxdf
import os
import numpy as np

#### Configurations

All the scripts using the `config.yaml` configuration file.
The first of each script is to load the config file and use
the parameters which relevant for the current part. The configuration
file arranged to be easy to read and change. For example,
we can get and set LabStreamingLayer parameters by:

In [ ]:
config = yaml.full_load(open('config.yaml', 'r'))
lsl_params = config['lsl']

print('\n'.join('{}: {}'.format(k, v) for k, v in lsl_params.items()))


The configuration file centralize all the interface parameters into
one file.

#### MI1 - Record Experiment

The `MI1_record_experiment.py` script responsible for the visualization
of the experiment and recording it. The main function `record_experiment()` get
paradigm (for now MI or SSVEP) and running experiment according given paradigm.
For now we only partially implemented the SSVEP paradigm and focused on MI.
We also implemented an interactive GUI which help the user for running an experiment.

The `MI_record()` function is the main function for recording a MI experiment.
The function create folder for the subject, initialize LSL stream,
creating random stimulus vector and start presenting the subject the images.
The function push before and after each image a marker to indicate the trial
start and end. Parameters for this part can be change in the
config file, such as the amount of trials, length of each trial, etc.

The output of `MI1` is xdf file which contains the EEG and markers record.
The second file is the stimulus vector which contains the order of the images
the subject saw in the experiment (i.e., the y vector).
The files' names are EEG.xdf and stimulus_vector.csv and both of the save in the subject directory.

#### MI2 - Data Pre-Processing

The `MI2_preprocess.py` script responsible for getting the subject's folder directory
from the config file and for each day of recording cleans the data.
Currently, the pre-processing script using low-pass and high-pass filters to clean the data.
The parameters for the filters are also part of the config
file under `config['preprocess']['filter']`.

